In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
40,application_1596730828793_0068,pyspark,idle,Link,Link


SparkSession available as 'spark'.

In [2]:
import boto3
import datetime
import pandas as pd
from hops import hdfs
from pyspark.sql import SparkSession
from pyspark.sql import Row

In [3]:
bucket_name = "iiaweb-s3-io-practice-bucket"

s3_resource = boto3.resource('s3', region_name='us-east-2')
bucket = s3_resource.Bucket(bucket_name)

Inspect bucket contents

In [ ]:
contents = list(bucket.objects.all())
zipped_filekeys = [object.key for object in contents if '.gzip' in object.key]
print(len(zipped_filekeys))

In [21]:
bucket_name = 'iiaweb-s3-io-practice-bucket'
key = '652/1/652200101092009.gzip'
print(bucket_name, key)

iiaweb-s3-io-practice-bucket 652/1/652200101092009.gzip

In [22]:
s3_client = boto3.client('s3')

# key = zipped_filekeys[0]
obj = s3_client.get_object(Bucket= bucket_name, Key= key)


print("Loading file: ", key)
pandas_df = pd.read_csv(obj['Body'], compression='gzip')
pandas_df.head()

Loading file:  652/1/652200101092009.gzip
             timestamp  FPAC  BLAC  CTAC  ...  EHRS_3  EHRS_2  ESN_1  ESN_2
0  2001-01-09 20:09:00     0     0     0  ...    6359    5633   7882   7833
1  2001-01-09 20:09:01     0     0     0  ...    6359    5633   7882   7833
2  2001-01-09 20:09:02     0     0     0  ...    6359    5633   7882   7833
3  2001-01-09 20:09:03     0     0     0  ...    6359    5633   7882   7833
4  2001-01-09 20:09:04     0     0     0  ...    6359    5633   7882   7833

[5 rows x 187 columns]

In [32]:
len(pandas_df)

3943

## Create Dictionary of Computation Times

In [42]:
times = {}

## Loading a Zipped CSV From S3 to Spark 

In [7]:
#Initialize a Master DF with the first file
started = datetime.datetime.now()

s3_client = boto3.client('s3')

# key = zipped_filekeys[0]

obj = s3_client.get_object(Bucket= bucket_name, Key= key)
df = spark.createDataFrame(pd.read_csv(obj['Body'], compression='gzip'))

finished = datetime.datetime.now()
elapsed = finished - started
print('Operation took: ', elapsed)

Operation took:  0:00:07.332341

In [8]:
df.show(5)

+-------------------+----+----+----+---+---+-----+-----+-----+-----+---+---+---+----+---+----+---+---+----+---+---+----+----+----+------+-------------+-----------------+-------------------+-------------------+----+----+---------+----+----+----+----+----+----+----+---+----+---+----+---+------------------+-------------------+---+-----------------+-------+----+---+-----+---------+----+----+----+----+-----+----+----+----+------+-----+--------------------+-------------------+-------+----+----+----+----+-----+------------------+------------------+------------------+--------+----------+-------+----+------+-----------------+-------------+---+---+----+-------------------+----+----+----+----+----+----+---+----+----+----+----+-----+-----+---------+---+-------------------+------+-----+-----+------+-----------------+-----------------+----+----+----+----+----+---+------+-----+----+---+----+----+----+----+----+-----+-----+--------------------+---+---------+-------------+-----------------+------------

## Hive

In [23]:
from hops import hdfs as hopsfs
PROJECT_NAME = hopsfs.project_name()

In [24]:
PROJECT_NAME

'MattTest'

## Show Tables

In [25]:
spark.sql("show tables").show()

+--------+-------------+-----------+
|database|    tableName|isTemporary|
+--------+-------------+-----------+
|matttest|nasa_dashlink|      false|
+--------+-------------+-----------+

In [20]:
df.show(5)

+-------------------+----+----+----+---+---+-----+-----+-----+-----+---+---+---+----+---+----+---+---+----+---+---+----+----+----+------+-------------+-----------------+-------------------+-------------------+----+----+---------+----+----+----+----+----+----+----+---+----+---+----+---+------------------+-------------------+---+-----------------+-------+----+---+-----+---------+----+----+----+----+-----+----+----+----+------+-----+--------------------+-------------------+-------+----+----+----+----+-----+------------------+------------------+------------------+--------+----------+-------+----+------+-----------------+-------------+---+---+----+-------------------+----+----+----+----+----+----+---+----+----+----+----+-----+-----+---------+---+-------------------+------+-----+-----+------+-----------------+-----------------+----+----+----+----+----+---+------+-----+----+---+----+----+----+----+----+-----+-----+--------------------+---+---------+-------------+-----------------+------------

## Create Initial Table

In [37]:
df.count()

3943

In [38]:
df.write.format("ORC").mode("overwrite").saveAsTable("NASA_Dashlink")

## Preview Data

In [39]:
spark.sql("Select count(*) from nasa_dashlink").show()

+--------+
|count(1)|
+--------+
|    3943|
+--------+

In [40]:
spark.sql("Select * from nasa_dashlink limit 10").show()

+-------------------+----+----+----+---+---+-----+-----+-----+-----+---+---+---+----+---+----+---+---+----+---+---+----+----+----+------+-------------+-----------------+-------------------+-------------------+----+----+---------+----+----+----+----+----+----+----+---+----+---+----+---+------------------+--------------------+-------+-----------------+-------+----+---+-----+---------+----+----+----+----+-----+----+----+----+------+-----+--------------------+-------------------+-------+----+----+----+----+-----+------------------+------------------+------------------+--------+----------+-------+----+------+-----------------+-------------+---+---+----+-------------------+----+----+----+----+----+----+---+----+----+----+----+-----+-----+---------+---+-------------------+------+-----+-----+------+-----------------+-----------------+----+----+----+----+----+---+------+-----+----+---+----+----+----+----+----+-----+-----+--------------------+---+---------+-----------------+-----------------+---